In [ ]:
import os
import re
import sys
import time
import datetime
import psycopg2
import numpy as np
import pandas as pd
import seaborn as sns

from scipy import stats
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
sys.path.insert(0, '../')
import cypy2

In [ ]:
from cycler import cycler
red, blue, green, purple, orange, yellow, brown, pink, gray  = sns.color_palette('Set1')
mpl.rcParams['axes.prop_cycle'] = cycler(color=[blue, orange, green, red, brown, gray])

In [ ]:
%matplotlib 
%load_ext autoreload
%autoreload 2

In [ ]:
user = 'keith'
host = 'localhost'
dbname = 'cypy2'
conn = psycopg2.connect(user=user, host=host, dbname=dbname)

In [ ]:
# 2018 horseshoe meadows road
a = cypy2.Activity.from_db(conn, '20180923163103')
a.metadata

In [ ]:
recs, recsp = a.records('raw'), a.records('proc')

### Histograms of adjacent timepoints

In [ ]:
offset = 1
column = 'power'
bins= {
    'cadence': np.arange(60, 90, 1),
    'power': np.arange(100, 300, 1),
    'speed': np.arange(5, 15, .1),
    'heart_rate': np.arange(120, 180, 1),
}
vals = recsp[column].values
vals = vals[~np.isnan(vals)]
h = plt.hist2d(vals[offset:], vals[:-offset], bins=bins[column])

### Autocorrelation functions

In [ ]:
def autocorr(x, max_offset, stepsize=1, drop_zeros=True):
    
    offsets = np.arange(1, max_offset, stepsize).astype(int)
    
    corrs = []
    for offset in offsets:
        vals = np.vstack((x[offset:], x[:-offset]))
        
        # drop nans
        vals = vals[:, ~np.isnan(vals).any(axis=0)]
        
        if drop_zeros:
            vals = vals[:, vals.all(axis=0)]
        
        cov = np.cov(vals[0, :], vals[1, :])
        corrs.append(cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1]))
        
    return offsets, corrs

In [ ]:
for col in ['heart_rate', 'power', 'cadence', 'speed']:
    plt.plot(*autocorr(recsp[col].values, 1000, 10, drop_zeros=True), marker='.', label=col)
plt.legend()
plt.gca().set_ylim([0, 1.1])

### Autoregressive models

In [ ]:
from sklearn import linear_model
from sklearn import model_selection

In [ ]:
wins = cypy2.utils.sliding_window(recs.power, 10, 10)
wins = wins[~np.isnan(wins).any(axis=1), :]
wins.shape

In [ ]:
train, test = model_selection.train_test_split(wins, train_size=.8)

In [ ]:
m = linear_model.LinearRegression()
m.fit(train[:, :-1], train[:, -1])
m.coef_

In [ ]:
yp = m.predict(wins[:, :-1])
plt.plot(wins[:, -1])
plt.plot(yp)